# Convergence on local interaction model


## Loading modules

In [ ]:
using QSWalk
using LightGraphs # for various 
using TikzGraphs # for graphs plot 

## Numerical proof of unique stationary state

Basic parameters. As graph we choose random Erdős–Rényi directed graph. Theory shows, that strongly connected graphs has unique stationary state. Note the Hamiltonian is chosen to be adjacency matrix of underlying graph.

In [ ]:
dim = 10
digraph = erdos_renyi(dim, 0.5, is_directed=true)
graph = Graph(digraph)
adjacencydigraph = full(adjacency_matrix(digraph))
adjacencygraph = full(adjacency_matrix(graph))
time = 100.
println("Is strongly connected: $(is_strongly_connected(digraph))")
TikzGraphs.plot(digraph)

Generating operators code.

In [ ]:
lind = classical_lindbladian(adjacencydigraph)
globaloperator = evolve_generator(adjacencygraph, lind);

The sufficient and necessary condition for quantum stochastic evolution being ergodic is one-dimensional null-space. This can be calulated as below. Note for large matrices *eigs* may be a better option.

In [ ]:
numspacedim = count(x->abs(x)<1e-5, eigvals(globaloperator))
println("Dimensionality of null-space of global operator: $numspacedim")

This allows efficient stationary state generation. Note that the trace may differ from one, as the eigenstate is normalized according to different norm.

In [ ]:
eigendecomposition = eigfact(globaloperator)
zeroindex = find(x -> abs(x)<=1.e-5, eigendecomposition[:values])
stationarystate = unres(vec(eigendecomposition[:vectors][:, zeroindex]))
println("Trace of stationary state $(trace(stationarystate))")
stationarystate /= trace(stationarystate)
println("Trace of stationary state $(trace(stationarystate))")

## Convergence

Since the stationary state is unique, all of the quantum states converges to it. We show this by taking three different states. Note, that for larger density states bigger time may need to be chosen.

In [ ]:
rhoinit1 = proj(1, dim)
rhoinit2 = proj(3, dim)
rhoinit3 = eye(dim)/dim;

Since we apply the same evolution for all of the initial states, it is more efficient to calulate exponent once.

In [ ]:
U = evolve_operator(globaloperator, time)
rho1 = evolve(U, rhoinit1)
rho2 = evolve(U, rhoinit2)
rho3 = evolve(U, rhoinit3);

In order to show those states are essentialy the same we can calulate difference nor

In [ ]:
println(norm(rho1-rho2))
println(norm(rho2-rho3))
println(norm(stationarystate-rho3))